# devlog 2024-10-09

_Author: Tyler Coles_

Methods for group-and-aggregate on time-series data.

epymorph output data can be thought of as three-dimensional -- the axes are 1. time, 2. geospatial node, 3. simulation data values (which can include simulation state and transition information; aka compartments and events). A very common requirement in data processing is to be able to summarize the time axis of this data. For example, computing the sum of infections over the whole time series, or a monthly maximum infection for every month during the simulation.

To determine the best approach, we generate some (suitably large) example data, group by calendar month, and compute the sum for each month. We compare by execution time and the simplicity of the solution.

In [1]:
from datetime import date
from epymorph.time import DateRange
import numpy as np
import pandas as pd
from calendar import monthrange

date_range = list(
    DateRange(
        date(2020, 1, 15),
        date(2030, 11, 24),
    )
)

days = len(date_range)
tau_steps = 2

dates = np.repeat(np.array(date_range, dtype=np.datetime64), repeats=tau_steps)
values = np.column_stack(
    [
        # This array structure simulates multiple values per day
        np.arange(days * tau_steps) + 0,  # e.g., this could be compartment S
        np.arange(days * tau_steps) + 1,  # this could be the S->I event
        np.arange(days * tau_steps) + 2,  # etc.
        # I call this dimension, generically, the number of "quantities"
    ]
)
quantities = values.shape[1]

print(f"{dates.shape=}")
print(f"{values.shape=}")
print([str(x) for x in dates[16:23]])
print(values[16:23, :])

dates.shape=(7932,)
values.shape=(7932, 3)
['2020-01-23', '2020-01-23', '2020-01-24', '2020-01-24', '2020-01-25', '2020-01-25', '2020-01-26']
[[16 17 18]
 [17 18 19]
 [18 19 20]
 [19 20 21]
 [20 21 22]
 [21 22 23]
 [22 23 24]]


In [2]:
# Utility functions


def count_months(d1: date, d2: date) -> int:
    """Compute the number of unique months included in a range of dates."""
    if d1 > d2:
        d1, d2 = d2, d1
    return (d2.year - d1.year) * 12 + (d2.month - d1.month) + 1


date_range_months = count_months(date_range[0], date_range[-1])
print(f"{date_range_months=}")


# epi weeks: https://www.cmmcp.org/mosquito-surveillance-data/pages/epi-week-calendars-2008-2024


def first_epi_day(year: int) -> pd.Timestamp:
    first_saturday = pd.Timestamp(year, 1, 1) + pd.offsets.Week(weekday=5)
    if first_saturday.day < 4:
        first_saturday = first_saturday + pd.offsets.Week(weekday=5)
    first_epi_day = first_saturday - pd.offsets.Week(weekday=6)
    return first_epi_day


def epi_week(check_date: date) -> tuple[int, int]:
    d = pd.Timestamp(check_date.year, check_date.month, check_date.day)
    last_year_day1 = first_epi_day(d.year - 1)
    this_year_day1 = first_epi_day(d.year)
    next_year_day1 = first_epi_day(d.year + 1)
    if d < this_year_day1:
        # in last years' epi weeks
        origin = last_year_day1
        year = d.year - 1
    elif d >= next_year_day1:
        # in next years' epi weeks
        origin = next_year_day1
        year = d.year + 1
    else:
        # in this years' epi weeks
        origin = this_year_day1
        year = d.year
    return year, (d - origin).days // 7 + 1


def epi_week_start(epi_week: tuple[int, int]) -> pd.Timestamp:
    year, week = epi_week
    day1 = first_epi_day(year)
    return day1 + pd.offsets.Week(n=week - 1)


# %timeit first_epi_day(2021)
# %timeit epi_week(date(2021, 1, 1))

date_range_months=131


## Task 1: computing monthly sums

### Approach 1: a basic Python loop

In [3]:
def calc_1():
    initial_year = date_range[0].year
    months = count_months(date_range[0], date_range[-1])
    result = np.zeros((months, quantities), dtype=values.dtype)
    for dnp, v in zip(dates, values):
        d = dnp.astype("object")
        t = (d.year - initial_year) * 12 + d.month - 1
        for q in range(quantities):
            result[t, q] += v[q]
    return result


%timeit calc_1()
res1 = calc_1()
res1[0:10, :]

14.2 ms ± 101 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


array([[  561,   595,   629],
       [ 3625,  3683,  3741],
       [ 7595,  7657,  7719],
       [11010, 11070, 11130],
       [15159, 15221, 15283],
       [18330, 18390, 18450],
       [22723, 22785, 22847],
       [26567, 26629, 26691],
       [29370, 29430, 29490],
       [34131, 34193, 34255]])

 ### Approach 2: Pandas resample

In [4]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#resampling
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects
# https://pandas.pydata.org/pandas-docs/stable/user_guide/cookbook.html#cookbook-resample


def calc_2():
    data_df = pd.DataFrame(index=pd.to_datetime(dates), data=values)
    result_df = data_df.resample(rule="MS").agg(func="sum")
    return result_df.to_numpy()


%timeit calc_2()
res2 = calc_2()
res2[0:10, :]

3 ms ± 23.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


array([[  561,   595,   629],
       [ 3625,  3683,  3741],
       [ 7595,  7657,  7719],
       [11010, 11070, 11130],
       [15159, 15221, 15283],
       [18330, 18390, 18450],
       [22723, 22785, 22847],
       [26567, 26629, 26691],
       [29370, 29430, 29490],
       [34131, 34193, 34255]])

### Approach 3: numpy, reshape to add a days-in-the-month axis, sum all at once

In [5]:
def calc_3():
    AGG_FN = np.sum
    ZERO = 0
    initial_date = date_range[0]
    final_date = date_range[-1]
    n = len(values)
    months = count_months(initial_date, final_date)
    matrix = np.full((months, 31 * tau_steps, quantities), ZERO, dtype=values.dtype)
    i, m = 0, 0
    while i < n:
        curr_date = dates[i].astype("object")
        _, month_len = monthrange(curr_date.year, curr_date.month)
        last_date = min(final_date, date(curr_date.year, curr_date.month, month_len))
        days_this_month = (last_date.day - curr_date.day) + 1
        j = i + (days_this_month * tau_steps)
        w = (curr_date.day - 1) * tau_steps
        z = w + days_this_month * tau_steps
        matrix[m, w:z, :] = values[i:j, :]
        i, m = j, m + 1
    return AGG_FN(matrix, axis=1)


%timeit calc_3()
res3 = calc_3()
res3[0:10, :]

420 μs ± 3.31 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


array([[  561,   595,   629],
       [ 3625,  3683,  3741],
       [ 7595,  7657,  7719],
       [11010, 11070, 11130],
       [15159, 15221, 15283],
       [18330, 18390, 18450],
       [22723, 22785, 22847],
       [26567, 26629, 26691],
       [29370, 29430, 29490],
       [34131, 34193, 34255]])

### Approach 4: numpy, compute sum for each month and accumulate

In [6]:
# numpy: month-by-month


def calc_4():
    initial_date = date_range[0]
    final_date = date_range[-1]
    n = len(values)
    months = count_months(initial_date, final_date)
    result = np.empty((months, quantities), dtype=values.dtype)
    i, m = 0, 0
    while i < n:
        curr_date = dates[i].astype("object")
        _, month_len = monthrange(curr_date.year, curr_date.month)
        last_date = min(final_date, date(curr_date.year, curr_date.month, month_len))
        days_this_month = (last_date.day - curr_date.day) + 1
        j = i + (days_this_month * tau_steps)
        result[m, :] = values[i:j, :].sum(axis=0)
        i, m = j, m + 1
    return result


%timeit calc_4()
res4 = calc_4()
res4[0:10, :]

678 μs ± 7.32 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


array([[  561,   595,   629],
       [ 3625,  3683,  3741],
       [ 7595,  7657,  7719],
       [11010, 11070, 11130],
       [15159, 15221, 15283],
       [18330, 18390, 18450],
       [22723, 22785, 22847],
       [26567, 26629, 26691],
       [29370, 29430, 29490],
       [34131, 34193, 34255]])

In [7]:
# Check that we get the same answer for all approaches.
(
    res1.shape == (date_range_months, quantities)
    and np.array_equal(res1, res2)
    and np.array_equal(res1, res3)
    and np.array_equal(res1, res4)
)

True

## Task 2: computing weekly sums

### Approach 1: pandas resample

This approach has the unfortunate side-effect of dropping partial weeks from the beginning and end of the time data timeframe.

In [8]:
def calc_weekly_pandas():
    data_df = pd.DataFrame(index=pd.to_datetime(dates), data=values)
    # Calc sum by weekday where weeks start on Sundays
    result_df = data_df.resample(rule=pd.offsets.Week(weekday=6)).agg(func="sum")
    # display(result_df)
    # Also you get the new dates automatically...
    return result_df.index, result_df.to_numpy()


%timeit calc_weekly_pandas()
calc_weekly_pandas()

7.51 ms ± 40.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


(DatetimeIndex(['2020-01-19', '2020-01-26', '2020-02-02', '2020-02-09',
                '2020-02-16', '2020-02-23', '2020-03-01', '2020-03-08',
                '2020-03-15', '2020-03-22',
                ...
                '2030-09-22', '2030-09-29', '2030-10-06', '2030-10-13',
                '2030-10-20', '2030-10-27', '2030-11-03', '2030-11-10',
                '2030-11-17', '2030-11-24'],
               dtype='datetime64[s]', length=567, freq='W-SUN'),
 array([[    45,     55,     65],
        [   231,    245,    259],
        [   427,    441,    455],
        ...,
        [110579, 110593, 110607],
        [110775, 110789, 110803],
        [ 95106,  95118,  95130]]))

### Approach 2: pandas groupby on week column

Pandas' resample function is powerful, but it only supports so many timeframe rules out of the box. "Epi week" is a very specific definition for the weeks in the year that attempts to standardize weekly comparisons. Maybe we could define a custom Pandas Offset class for this, but it's simpler to use a column which describes the group each row belongs to:

In [9]:
def calc_epiweekly_pandas():
    data_df = pd.DataFrame(index=pd.to_datetime(dates), data=values)
    data_df["epiweek"] = data_df.index.map(epi_week)
    # Calc sum by epi week
    result_df = data_df.groupby("epiweek").agg(func="sum")
    # Also we can also return the new dates...
    return result_df.index.map(epi_week_start), result_df.to_numpy()


%timeit calc_epiweekly_pandas()
calc_epiweekly_pandas()

673 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


(DatetimeIndex(['2020-01-12', '2020-01-19', '2020-01-26', '2020-02-02',
                '2020-02-09', '2020-02-16', '2020-02-23', '2020-03-01',
                '2020-03-08', '2020-03-15',
                ...
                '2030-09-15', '2030-09-22', '2030-09-29', '2030-10-06',
                '2030-10-13', '2030-10-20', '2030-10-27', '2030-11-03',
                '2030-11-10', '2030-11-17'],
               dtype='datetime64[ns]', name='epiweek', length=567, freq=None),
 array([[    28,     36,     44],
        [   203,    217,    231],
        [   399,    413,    427],
        ...,
        [110551, 110565, 110579],
        [110747, 110761, 110775],
        [110943, 110957, 110971]]))

## Conclusion:

While the numpy versions are somewhat faster, they are much more challenging to write and need to be specially designed to deal with different time periods. It's not obvious how to adjust the logic to work with arbitrary time periods like epi weeks.

On the other hand, Pandas is not tragically slow -- faster than I was expecting -- and it's very obvious how to extend it for highly arbitrary time period definitions. Pandas resample behavior is a little esoteric, but adding a column on which to group is very simple. That's a clear winner in this situation.

Note: I also tested using polars for this as a curiosity. It was about 10x faster, but didn't produce the same results.

## Appendix: unit tests for the epi week functions

In [10]:
import unittest


class EpiWeeksTest(unittest.TestCase):
    def test_first_epi_day(self):
        self.assertEqual(first_epi_day(2020), pd.Timestamp(2019, 12, 29))
        self.assertEqual(first_epi_day(2021), pd.Timestamp(2021, 1, 3))
        self.assertEqual(first_epi_day(2022), pd.Timestamp(2022, 1, 2))
        self.assertEqual(first_epi_day(2023), pd.Timestamp(2023, 1, 1))
        self.assertEqual(first_epi_day(2024), pd.Timestamp(2023, 12, 31))
        self.assertEqual(first_epi_day(2025), pd.Timestamp(2024, 12, 29))

    def test_epi_week(self):
        self.assertEqual(epi_week(date(2021, 1, 1)), (2020, 53))
        self.assertEqual(epi_week(date(2021, 1, 2)), (2020, 53))
        self.assertEqual(epi_week(date(2021, 1, 3)), (2021, 1))

        self.assertEqual(epi_week(date(2024, 1, 1)), (2024, 1))
        self.assertEqual(epi_week(date(2024, 1, 6)), (2024, 1))
        self.assertEqual(epi_week(date(2024, 1, 7)), (2024, 2))
        self.assertEqual(epi_week(date(2024, 3, 14)), (2024, 11))
        self.assertEqual(epi_week(date(2024, 12, 28)), (2024, 52))
        self.assertEqual(epi_week(date(2024, 12, 29)), (2025, 1))
        self.assertEqual(epi_week(date(2024, 12, 31)), (2025, 1))


# Run tests here in the notebook
suite = unittest.TestSuite()
suite.addTests(unittest.TestLoader().loadTestsFromTestCase(EpiWeeksTest))
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>